# System Checker

## Load System Data

In [60]:
import MDAnalysis as mda
from MDAnalysis.analysis.leaflet import optimize_cutoff
from MDAnalysis.analysis.leaflet import LeafletFinder
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import pandas as pd

In [2]:
prefix = "../Sample_files/TRPV2_test/" #File location
top_file = f"{prefix}step7_production.tpr" #Topology file
top_file_2 = f"{prefix}step6.6_equilibration.gro" #Topology file 2
index = f"{prefix}index.ndx" #Index file

lipid_list = ["CHOL", "POPC", "PAPC", "PAPE", "DPSM", "PNGS", "POPA", "PAP1", "PAP6"] #List of lipids in the system

## Check membrane: leaflet lipid composition, charges

In [6]:
#Load data as a MDA universe object
u = mda.Universe(top_file, top_file_2)

In [116]:
cutoff,N = optimize_cutoff(u, "name PO4 ROH")
L = LeafletFinder(u, 'name PO4 ROH AM1', cutoff=cutoff)
upper_resids = " ".join({str(i) for i in L.groups(0).resids})
lower_resids = " ".join({str(i) for i in L.groups(1).resids})

upper_atoms = u.select_atoms(f"resid {upper_resids}")
lower_atoms = u.select_atoms(f"resid {lower_resids}")

upper_lipids = {l:len(np.unique(L.groups(0)[L.groups(0).resnames == l].resids)) for l in np.unique(L.groups(0).resnames)}
lower_lipids = {l:len(np.unique(L.groups(1)[L.groups(1).resnames == l].resids)) for l in np.unique(L.groups(1).resnames)}

upper_ratios = {l:value/np.sum(list(upper_lipids.values())) for l,value in upper_lipids.items()}
lower_ratios = {l:value/np.sum(list(lower_lipids.values())) for l,value in lower_lipids.items()}

upper_ratios_2 = {l:value/np.min(list(upper_lipids.values())) for l,value in upper_lipids.items()}
lower_ratios_2 = {l:value/np.min(list(lower_lipids.values())) for l,value in lower_lipids.items()}

ul_charge = upper_atoms.total_charge()
ll_charge = lower_atoms.total_charge()

ul_ind_charge = {lipid:upper_atoms[upper_atoms.resnames == lipid].total_charge() for lipid in upper_lipids}
ll_ind_charge = {lipid:lower_atoms[lower_atoms.resnames == lipid].total_charge() for lipid in lower_lipids}

ul_charge_per_lipid = {lipid:ul_ind_charge[lipid]/upper_lipids[lipid] for lipid in upper_lipids}
ll_charge_per_lipid = {lipid:ll_ind_charge[lipid]/lower_lipids[lipid] for lipid in lower_lipids}

In [121]:
df = pd.DataFrame.from_dict([upper_lipids,upper_ratios, upper_ratios_2, ul_charge_per_lipid, ul_ind_charge,\
                        lower_lipids,lower_ratios, lower_ratios_2, ll_charge_per_lipid, ll_ind_charge]).T

df.columns = ["Upper Leaflet Count", "Upper Leaflet Ratio", "Upper Leaflet Ratio (2)", "Charge per lipid U", "Upper Leaflet Charge",
             "Lower Leaflet Count", "Lower Leaflet Ratio", "Lower Leaflet Ratio (2)", "Charge per lipid L", "Lower Leaflet Charge"]
df.loc["Total"] = df.sum()

df

,Upper Leaflet Count,Upper Leaflet Ratio,Upper Leaflet Ratio (2),Charge per lipid U,Upper Leaflet Charge,Lower Leaflet Count,Lower Leaflet Ratio,Lower Leaflet Ratio (2),Charge per lipid L,Lower Leaflet Charge
CHOL,264.0,0.413146,7.135135,0.0,0.0,212.0,0.339744,42.4,0.0,0.0
DPSM,75.0,0.117371,2.027027,0.0,0.0,10.0,0.016026,2.0,0.0,0.0
PAPC,37.0,0.057903,1.000000,0.0,0.0,5.0,0.008013,1.0,0.0,0.0
PAPE,75.0,0.117371,2.027027,0.0,0.0,60.0,0.096154,12.0,0.0,0.0
PNGS,75.0,0.117371,2.027027,0.0,0.0,NaN,NaN,NaN,NaN,NaN
POPC,113.0,0.176839,3.054054,0.0,0.0,65.0,0.104167,13.0,0.0,0.0
PAP1,NaN,NaN,NaN,NaN,NaN,20.0,0.032051,4.0,-3.0,-60.0
PAP6,NaN,NaN,NaN,NaN,NaN,20.0,0.032051,4.0,-5.0,-100.0
PAPS,NaN,NaN,NaN,NaN,NaN,212.0,0.339744,42.4,-1.0,-212.0
POPA,NaN,NaN,NaN,NaN,NaN,20.0,0.032051,4.0,-2.0,-40.0


{'CHOL': 0.0, 'DPSM': 0.0, 'PAPC': 0.0, 'PAPE': 0.0, 'PNGS': 0.0, 'POPC': 0.0}